In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data_re_id.npy');
path_to_labels = path.join(root_folder,'data/train_labels_re_id.npy');
#path_to_mean_std = path.join(root_folder,'data/train_data/synthesized/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [39]:
#data = np.load(path_to_train_data);
#labels = np.load(path_to_labels);

torch_labels = torch.tensor(np.matlib.repmat(np.array([0,1,2, 3, 4, 5, 6,7, 8,9, 10]), 1, 500)).int()
torch_data = torch.zeros((torch_labels.nelement(), 1 , 72)).int()

for i in range(torch_data.size()[0]):
    torch_data[i, :] = torch_data[i, :] + i;
nb_of_classes = torch_labels.unique().size()[0]
if (len(torch_labels.size()) == 2):
    torch_labels = torch.squeeze(torch_labels)
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


In [ ]:
for i in range(torch_labels.nelement()):
    for j in range(test_data.data.size()[0]):
        res  = (torch_data[i, :] == test_data.data[j, :].int()).numpy()
        if(np.sum(res) > 0):
            print(torch_labels[i] == test_data.labels[j].int(),torch_labels[i], test_data.labels[j])


tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8)

tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(1, dtype=torch.int32) tensor(1.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(9, dtype=torch.int32) tensor(9.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(9, dtype=torch.int32) tensor(9.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(9, dtype=torch.int32) tensor(9.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) ten

tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(1, dtype=torch.int32) tensor(1.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(1, dtype=torch.int32) tensor(1.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(1, dtype=torch.int32) tensor(1.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) ten

tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) ten

tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(9, dtype=torch.int32) tensor(9.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) t

tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(6, dtype=torch.int32) tensor(6.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(1, dtype=torch.int32) tensor(1.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) t

tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(9, dtype=torch.int32) tensor(9.)
tensor(1, dtype=torch.uint8) tensor(10, dtype=torch.int32) tensor(10.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(7, dtype=torch.int32) tensor(7.)
tensor(1, dtype=torch.uint8)

tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(8, dtype=torch.int32) tensor(8.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(3, dtype=torch.int32) tensor(3.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.int32) tensor(0.)
tensor(1, dtype=torch.uint8) tensor(5, dtype=torch.int32) tensor(5.)
tensor(1, dtype=torch.uint8) tensor(1, dtype=torch.int32) tensor(1.)
tensor(1, dtype=torch.uint8) tensor(2, dtype=torch.int32) tensor(2.)
tensor(1, dtype=torch.uint8) tensor(4, dtype=torch.int32) tensor(4.)


In [42]:
train_data.__len__()

4400